In [ ]:
import pandas as pd
import json
import glob
import os
import math
import csv
from csv import writer
from csv import reader

#input file directory
main_directory='C:\\Users\\CASALAB\\Desktop\\progetto_CASA\\'

k=0
y=0
#threshold of the number of key points
keypoint_threshold = 12
#threshold of the accuracy of key points
accuracy_threshold = 0.7
#CSV header list
header_list = ["Frame","Face","keypoint","Body","1_x","1_y","1_accuracy","2_x","2_y","2_accuracy","3_x","3_y","3_accuracy","4_x","4_y","4_accuracy","5_x","5_y","5_accuracy","6_x","6_y","6_accuracy","7_x","7_y","7_accuracy","8_x","8_y","8_accuracy","9_x","9_y","9_accuracy",
               "10_x","10_y","10_accuracy","11_x","11_y","11_accuracy","12_x","12_y","12_accuracy","13_x","13_y","13_accuracy","14_x","14_y","14_accuracy","15_x","15_y","15_accuracy","16_x","16_y","16_accuracy","17_x","17_y","17_accuracy","18_x","18_y","18_accuracy","19_x","19_y","19_accuracy",
               "20_x","20_y","20_accuracy","21_x","21_y","21_accuracy","22_x","22_y","22_accuracy","23_x","23_y","23_accuracy","24_x","24_y","24_accuracy","25_x","25_y","25_accuracy"]
#face default value
face='NO'
#open the file with values about face
dlib = pd.read_csv('dlib.csv', names=["Frame", "Face","x1","y1","x2","y2"])
list_face=list(dlib["Face"])
list_name_frame=list(dlib["Frame"])

#open the output file in write mode
with open('merged_json_csv.csv', mode='w', newline='') as jsonfile:
    json_writer = csv.writer(jsonfile)
    entries = os.listdir(main_directory)
    #open every json file in the input directory
    for f in entries:
        read_files = glob.glob(main_directory + f + '/*.json')
        frame = None
        for d in read_files:
            with open(d, "rb") as data_file:
                data = json.load(data_file)
                json_struct = pd.read_json(d)
                #select the subject in json file with the highest backbone dimension using openpose keypoints
                if json_struct.shape[0] > 1:
                    max_distance = -1
                    num_of_elements = json_struct.shape[0]
                    best = 1
                    for i in range(num_of_elements):
                        if data['people'][i - 1]['pose_keypoints_2d'][3] != 0 and \
                                data['people'][i - 1]['pose_keypoints_2d'][24] != 0:
                            p1 = [data['people'][i - 1]['pose_keypoints_2d'][3],
                                  data['people'][i - 1]['pose_keypoints_2d'][4]]
                            p2 = [data['people'][i - 1]['pose_keypoints_2d'][24],
                                  data['people'][i - 1]['pose_keypoints_2d'][25]]
                            distance = math.sqrt(((p1[0] - p2[0]) ** 2) + ((p1[1] - p2[1]) ** 2))
                            if max_distance < distance:
                                max_distance = distance
                                best = i
                    frame = data['people'][best - 1]['pose_keypoints_2d']
                elif json_struct.shape[0] == 1:
                    frame = data['people'][0]['pose_keypoints_2d']

            # Determines whether the frame is good or not based on selected thresholds and writes it to the csv file
            if frame is not None:
                i = 0
                detected_keypoints = 0
                while i < len(frame):
                    x = frame[i]
                    y = frame[i + 1]
                    acc = frame[i + 2]
                    i += 3
                    if x != 0 and y != 0 and acc > accuracy_threshold:
                        detected_keypoints += 1
    
                # set the label value 
                label = 'Good' if detected_keypoints >= keypoint_threshold else 'Bad'
                filename = d.replace(main_directory, '').replace('_keypoints.json', '')
                
                #set the value of the face presence
                for y in range(len(list_name_frame)):
                    if list_name_frame[y]==filename[7:] or list_name_frame[y]==filename[6:]:
                        if list_face[y]==1:
                            face='YES'
                        else:
                            face='NO'
                    
                #write the csv row            
                json_writer.writerow(
                    [filename] +[face]+
                    [detected_keypoints] +
                    [label] +
                    frame
                )
#open the output file
df = pd.read_csv(main_directory + 'merged_json_csv.csv', names=header_list)
df